In [ ]:
from notebook_prelude import *
from relabeling import coreference

In [ ]:
embeddings = glob('data/embeddings/graph-embeddings/*.npy')

embedding_data = {}
for embedding in embeddings:
    filename = embedding.split('/')[-1].replace('.label-lookup.npy', '')
    with open(embedding, 'rb') as f:
        embedding_data[filename] = pickle.load(f)

In [ ]:
data = collections.defaultdict(lambda: collections.defaultdict(dict))
for filename, lookup in embedding_data.items():
    dataset, threshold, topn = re.findall(r'(.+?)\.threshold-(.+?)\.topn-(.+)', filename)[0]
    cliques = coreference.get_cliques_from_lookup(lookup)
    clique_sizes = [len(x) for x in cliques.values()]
    data[dataset][topn][threshold] = clique_sizes

In [ ]:
for dataset, d in data.items():
    for topn, thresholds in d.items():
        num_items = len(thresholds.keys())
        fig, ax = plt.subplots()
        for threshold, clique_size in thresholds.items():
            s = pd.Series(clique_size)
            # Remove outliers
            s = s[s < (s.quantile(0.98))]
            s.plot(kind='hist', alpha=0.8, label='threshold={}'.format(threshold), ax = ax, bins=60, logy=True)
        ax.set_title('dataset={}, topn={}'.format(dataset, topn))
        ax.set_xlabel('clique size')
        ax.legend()
        fig.tight_layout()
        plt.show()
        plt.close(fig)